<a href="https://colab.research.google.com/github/shahyash07/Credit-card-fraud-detection-project/blob/master/creditcar_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# credit card fraud detection


random forest, logistic regression and naive 
Bayes machine learning methods and their implementation

Performance of the models are compared using k-fold cross validation for different k values to choose the best model. 





In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import sklearn as sk

import warnings  # ignoring the warnings
warnings.filterwarnings('ignore')

In [ ]:
# importing data set from drive
from google.colab import drive
drive.mount('/content/gdrive')

input_path = '/content/gdrive/My Drive/'
# reading the data (target)
target_dataset = pd.read_csv(input_path + 'default of credit card clients.csv')

target_dataset.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [ ]:
print("explaining wthe columns") 
print(target_dataset.iloc[0])
target_dataset = target_dataset.drop([0], axis=0)

explaining wthe columns
ID                             ID
X1                      LIMIT_BAL
X2                            SEX
X3                      EDUCATION
X4                       MARRIAGE
X5                            AGE
X6                          PAY_0
X7                          PAY_2
X8                          PAY_3
X9                          PAY_4
X10                         PAY_5
X11                         PAY_6
X12                     BILL_AMT1
X13                     BILL_AMT2
X14                     BILL_AMT3
X15                     BILL_AMT4
X16                     BILL_AMT5
X17                     BILL_AMT6
X18                      PAY_AMT1
X19                      PAY_AMT2
X20                      PAY_AMT3
X21                      PAY_AMT4
X22                      PAY_AMT5
X23                      PAY_AMT6
Y      default payment next month
Name: 0, dtype: object


### Separating features from output variable
The dataframe of the target dataset has a column we would like to predict its values (output variable). We need to separate this column from the rest of the dataframe which include the features we want to use to build the model.

In [ ]:
output_var = target_dataset['Y']
input_features = target_dataset[[col for col in target_dataset.columns if 'X' in col]]
print('number of features: {}'.format(input_features.shape[0]))

target = target_dataset['Y']
features = target_dataset.drop(columns={'Y'})

number of features: 30000


In [ ]:
target.shape
features.shape

(30000, 24)

### Checking balance of classes
We need to determine if there is a class imbalance in the dataset as it will be important for choosing the metric for the performance assessment.

In [ ]:
from collections import Counter    # checking the count for 1 and zero in the target variable

Counter(target)        

Counter({'0': 23364, '1': 6636})

In [ ]:
# splitinng the data for test train split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.30, random_state=5)

In [ ]:
# model generation
from sklearn.model_selection import cross_val_score   # for k fold cross validation
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier 


In [ ]:
# Create logistic regression object
rf = RandomForestClassifier(random_state=10)

# Create logistic regression object
logreg = LogisticRegression(random_state=10)

# Create naive Bayes object
nb = GaussianNB()


In [ ]:
# scores_rf = cross_val_score(rf, X_train, y_train, cv=10)

In [ ]:
# q = cross_val_score()
# scores_rf

In [ ]:

# assessing performance of the model using 10-fold cross validation

# considering f1 scores 
scores_rf = cross_val_score(rf, X_train, y_train, cv=5,scoring='f1_macro' )

scores_logreg = cross_val_score(logreg, X_train, y_train, cv=5, scoring='f1_macro')

scores_nb = cross_val_score(nb, X_train, y_train, cv=5, scoring='f1_macro')

In [ ]:
scores_rf

array([0.69012057, 0.67759126, 0.67372507, 0.6928195 , 0.66762124])

In [ ]:
# average performance across all folds
print("Average Accuracy of random forest model across the folds: %0.2f" % (scores_rf.mean()))
print("Average Accuracy of logistic regression model across the folds: %0.2f" % (scores_logreg.mean()))
print("Average Accuracy of naive Bayes model across the folds: %0.2f" % (scores_nb.mean()))

Average Accuracy of random forest model across the folds: 0.68
Average Accuracy of logistic regression model across the folds: 0.44
Average Accuracy of naive Bayes model across the folds: 0.38


As random forest has higher performance in the cross-validation setting, we use it in the test set to further assess its performance.  

In [ ]:
# Let's train the model using all the training set together
rf.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_test = rf.predict(X_test)

In [ ]:
from sklearn import metrics

print("\n\ncomparing balanced accuracies of the models using the test set")
print("blanced accuracy of the predictions using random forest:", metrics.balanced_accuracy_score(y_test, y_pred_test))
print("recall of the predictions using random forest:", metrics.f1_score(y_test, y_pred_test, average='macro'))



comparing balanced accuracies of the models using the test set
blanced accuracy of the predictions using random forest: 0.653035178368642
recall of the predictions using random forest: 0.6767280177623852


# Tuning the model

In [ ]:
# parameters used in the base model 
rf.get_params()


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 10,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# using random searchcv 
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(round(x)) for x in np.linspace(10, 110, num = 6)]
max_depth.append(None)
# Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]/
# Method of selecting samples for training each tree
# bootstrap = [True, False]
# Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
random_grid = {'n_estimators': n_estimators, 
               'max_depth': max_depth,
            }

In [ ]:
# Use the random grid to search for best hyperparameters

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 35 candidates, totalling 105 fits
[CV] n_estimators=200, max_depth=10 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... n_estimators=200, max_depth=10, total=   5.2s
[CV] n_estimators=200, max_depth=10 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.2s remaining:    0.0s


[CV] ................... n_estimators=200, max_depth=10, total=   5.2s
[CV] n_estimators=200, max_depth=10 ..................................
[CV] ................... n_estimators=200, max_depth=10, total=   5.2s
[CV] n_estimators=650, max_depth=10 ..................................
[CV] ................... n_estimators=650, max_depth=10, total=  16.7s
[CV] n_estimators=650, max_depth=10 ..................................
[CV] ................... n_estimators=650, max_depth=10, total=  16.6s
[CV] n_estimators=650, max_depth=10 ..................................
[CV] ................... n_estimators=650, max_depth=10, total=  16.6s
[CV] n_estimators=1100, max_depth=10 .................................
[CV] .................. n_estimators=1100, max_depth=10, total=  28.0s
[CV] n_estimators=1100, max_depth=10 .................................
[CV] .................. n_estimators=1100, max_depth=10, total=  28.1s
[CV] n_estimators=1100, max_depth=10 .................................
[CV] .

[Parallel(n_jobs=1)]: Done 105 out of 105 | elapsed: 76.8min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rf_random.best_params_  #  checking the best parameters

{'max_depth': 10, 'n_estimators': 1550}

In [ ]:
# defining function to check the accuracy

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
base_model = RandomForestClassifier(random_state = 10)
base_model.fit(X_train,y_train)
# base_accuracy = evaluate(base_model, X_test, y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [ ]:
best_random = rf_random.best_estimator_
# random_accuracy = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
# now applying gridseach based on the best parameters found from the random searchcv 


from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [5,10,15],
    'n_estimators': [1500,1550,1600]
}

# {'max_depth': 10, 'n_estimators': 1550}

In [ ]:
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)


In [ ]:
# X_test

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train,y_train)
grid_search.best_params_



Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 13.0min finished


{'max_depth': 10, 'n_estimators': 1500}

In [ ]:
best_grid = grid_search.best_estimator_
# grid_accuracy = evaluate(best_grid, X_test, y_test)/

NameError: ignored